In [5]:
# %pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import warnings
from tqdm import tqdm

# SSL 인증서 경고 무시
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

# FAQ 페이지 URL
BASE_URL = "https://accident.knia.or.kr/faq"

# 결과를 저장할 리스트
faq_data = []

try:
    # 페이지 요청 (SSL 검증 무시)
    response = requests.get(BASE_URL, verify=False)
    response.raise_for_status()  # 오류 발생 시 예외 발생
    
    bs = BeautifulSoup(response.text, 'html.parser')
    
    faq_q_frms = bs.find_all('div', class_='faq_q_frm')
    print(f"총 {len(faq_q_frms)}개의 질문")
    faq_a_frms = bs.find_all('div', class_='faq_a_frm')
    print(f"총 {len(faq_a_frms)}개의 답변")
    
    # 답변 id 매핑
    faq_a_dict = {a.get('id'): a for a in faq_a_frms}
    
    for q in tqdm(faq_q_frms, desc="크롤링 진행 중"):
        # 질문 텍스트 추출
        question_div = q.find('div', class_='faq_txt')
        question_text = question_div.get_text(strip=True) if question_div else ''
        
        # 질문에 연결된 답변 id 추출
        link = question_div.find('a') if question_div else None
        answer_id = None
        if link and 'javascript:menuclick' in link.get('href', ''):
            answer_id = link.get('href').split('(')[-1].strip(');')
        
        # 답변 텍스트 추출
        answer_texts = []
        if answer_id and answer_id in faq_a_dict:
            answer_div = faq_a_dict[answer_id].find('div', class_='faq_txt')
            
            if answer_div:
                p_tags = answer_div.find_all('p')

                for p in p_tags:
                    text = p.get_text(strip=True)
                    
                    if text:
                        answer_texts.append(text)
        
        # 질문과 답변이 모두 있는 경우만 결과에 추가
        if question_text and answer_texts:
            faq_data.append({'question': question_text, 'answer': answer_texts})

except Exception as e:
    print(f"크롤링 중 오류 발생 : {str(e)}")



# JSON 파일 저장
output_filename = 'faq.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(faq_data, f, ensure_ascii=False, indent=2)

print(f"\nFAQ 데이터가 '{output_filename}' 파일로 저장되었습니다. (총 {len(faq_data)}개 항목)")


총 62개의 질문
총 62개의 답변


크롤링 진행 중: 100%|██████████| 62/62 [00:00<00:00, 14850.49it/s]


FAQ 데이터가 'faq.json' 파일로 저장되었습니다. (총 61개 항목)
